## imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

In [8]:
#!pip install dask

## extract data from gcp

In [11]:
#this does not group by tariff type
def process_group_data(g):
    #not required for Chris
    df=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_v1.csv")
    df.drop(columns='Unnamed: 0',inplace=True)
    #imputing zeros and nans on group avverage as opposed to per house for speed
    df1=df.loc[:,['LCLid','Acorn_Group','DateTime','KWH/hh','Tariff']]
    df1.loc[:,'DateTime']=pd.to_datetime(df1['DateTime'])
    #group average
    df2=df1.groupby(by=df.DateTime).mean()
    df2.sort_index(inplace=True)
    #fill 0s with nans
    df2['KWH/hh'].replace(0,np.nan,inplace=True)
    #fill nans for both existing ones and any created due to missing HH 
    date_range = pd.DataFrame(pd.date_range(df2.index[0],df2.index[-1], freq='30 min'),columns=['DateTime'])
    df2.reset_index(inplace=True)
    df2.loc[:,'DateTime']=pd.to_datetime(df2['DateTime'])
    df3=date_range.merge(df2,on='DateTime',how='outer')
    if np.sum(df3['KWH/hh'].isna())!=0:
        df3.fillna(method='bfill',inplace=True)
    return df3

## Check start and end date for each group

In [3]:
groups=['A','B','C','D','F','G','H','I','J','K','L','M','N','O','P','Q']
def extract_start_end_time_bygroup(groups):
    g_dict={}
    g_dict['E']=[['2011-11-24 11:00:00', '2014-02-28 00:00:00']]
    for g in groups:
        df=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_v1.csv",usecols=['DateTime','KWH/hh'])
        df.set_index('DateTime',inplace=True)
        df_av=df.groupby(by=df.index).mean()
        df_av.sort_index(inplace=True)
        g_dict[g]=[[df_av.index[0],df_av.index[-1]]]
        
    return g_dict

In [ ]:
# df=pd.read_csv('https://storage.googleapis.com/energy_usage_prediction_903/df_E_v1.csv',usecols=['DateTime','KWH/hh'])
# df.set_index('DateTime',inplace=True)
# t=[df.index[0],df.index[-1]]#['2012-07-09 11:30:00', '2014-02-28 00:00:00']

In [10]:
# #read data in chunks of 1 million rows at a time
# chunk = pd.read_csv('https://storage.googleapis.com/energy_usage_prediction_903/df_E_v1.csv',chunksize=1000000)
# pd_df = pd.concat(chunk)

In [32]:
groups=['A','B','C','D','F','G','H','I','J','K','L','M','N','O','P','Q']
def merge_groups(groups,use_tariff=True):
    if use_tariff==True:
        df = pd.DataFrame(columns=['DateTime','KWH/hh','Acorn_Group'])
        
    
    for g  in groups:
        if use_tariff==True:
            df1=pd.read_csv(f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_avg_v1.csv").drop(columns='Unnamed: 0')
        
        df1['Acorn_Group']=g
        df=pd.concat([df, df1])
    return df


In [29]:
https://storage.googleapis.com/energy_usage_prediction_903/df_A_avg_Std_v1.csv
https://storage.googleapis.com/energy_usage_prediction_903/df_A_avg_ToU_v1.csv

In [34]:
f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_avg_v1.csv"

'https://storage.googleapis.com/energy_usage_prediction_903/df_Q_avg_v1.csv'

In [39]:
f"https://storage.googleapis.com/energy_usage_prediction_903/df_{g}_avg{'_Std'}_v1.csv"

'https://storage.googleapis.com/energy_usage_prediction_903/df_Q_avg_Std_v1.csv'